This notebook collects all the game ID's from 

In [56]:
import json
from urllib.request import urlopen
import pandas as pd
import os
from tqdm.notebook import tqdm

# variables
scrape_revies = True # if data should be scraped or loaded from csv
reviews2scrap = 100 # number of reviews to scrape, if 0 all reviews will be scraped
all_games_url = "http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key=STEAMKEY&format=json"
game_detail_url = "http://store.steampowered.com/api/appdetails?appids="
review_url = 'https://store.steampowered.com/appreviews/'


In [58]:
# Scraping the data

if scrape:
    # get all app ID
    with urlopen(all_games_url) as url:
        df = pd.json_normalize(json.load(url)["applist"], record_path =['apps'])
        
    # add review column
    df["review_score"] = ""
    df["num_reviews"] = ""
    
    # check if all should be scraped
    if reviews2scrap == 0: reviews2scrap = len(df)
    
    # scrap first reviews2scrap reviews
    for appid in tqdm(df.head(reviews2scrap)["appid"]):
        # create specific review url
        with urlopen(review_url + str(appid) + "?json=1") as url:
            # append review score
            temp = json.load(url)
            df.loc[df.appid == appid, "review_score"] = temp["query_summary"]["review_score"]
            df.loc[df.appid == appid, "num_reviews"] = temp["query_summary"]["num_reviews"]

    df.to_csv('raw_data.csv')

else:
    df = pd.read_csv('raw_data.csv', index_col=0)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Cleaning the data
# games that have no reviews
df.drop(df[df.num_reviews == 0].index, inplace=True)

# games for which information was not fetched
df.drop(df[df.num_reviews == ""].index, inplace=True)
df.drop(df[df.review_score == ""].index, inplace=True)
df.to_csv('clean_data.csv')
